# Exploring Terms in the Encyclopaedia Britannica

## Similar terms within an edition - BERT - Part II


https://theaidigest.in/how-to-do-semantic-document-similarity-using-bert/

### Loading the necessary libraries

In [163]:
import yaml
import matplotlib.pyplot as plt
import numpy as np
import collections
import matplotlib as mpl

In [164]:
import networkx as nx
import matplotlib.pyplot as plt

In [165]:
import os
import pandas as pd

In [166]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

### Functions

In [167]:
def calculating_similarity_text_df(definition, text_embeddings):
    definition_embedding= model.encode(definition, batch_size = 8, show_progress_bar = True)
    similarity_def=cosine_similarity( [definition_embedding], text_embeddings)
    similarity_def_sorted = similarity_def.argsort()
    id_1 = []
    id_2 = []
    score = []
    for index,array in enumerate(similarity_def_sorted):
        id_1.append(index)
        score.append(similarity_def[index][array[-1]])
        id_2.append(array[-1])
    index_df = pd.DataFrame({'id_1' : id_1,
                          'id_2' : id_2,
                          'score' : score})
    return index_df

In [168]:
def calculating_similarity_text(definition, text_embeddings):
    definition_embedding= model.encode(definition, batch_size = 8, show_progress_bar = True)
    similarities=cosine_similarity( [definition_embedding], text_embeddings)
    similarities_sorted = similarities.argsort()
    results={}
    for i in range(-1, -12, -1):
        similar_index=similarities_sorted[0][i]
        rank=similarities[0][similar_index]
        results[uris[similar_index]]=[terms_info[similar_index][1],terms_info[similar_index][2], terms_info[similar_index][4], terms_info[similar_index][0], documents[similar_index], rank]
    return results
    
 
   

In [169]:
def get_document(uri):
    uri="<"+uri+">"
    sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
    query="""
    PREFIX eb: <https://w3id.org/eb#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?definition ?term
        WHERE {{
            %s a eb:Article ;
               eb:name ?term ;
               eb:definition ?definition . 
            }
            UNION {
            %s a eb:Topic ;
              eb:name ?term ; 
              eb:definition ?definition . 
            }
       } 
    """ %(uri, uri)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    term = results["results"]["bindings"][0]["term"]["value"]
    definition=results["results"]["bindings"][0]["definition"]["value"]
    return term, definition

## We have dataframe with these columns

- definition:           Definition of a term
- editionNum:           1,2,3,4,5,6,7,8
- editionTitle:         Title of the edition
- header:               Header of the page's term                                  
- place:                Place where the volume was edited (e.g. Edinburgh)                                    
- relatedTerms:         Related terms (see X article)  
- altoXML:              File Path of the XML file from which the term belongs       
- term:                 Term name                            
- positionPage:         Position of ther term in the page     
- startsAt:             Number page in which the term definition starts 
- endsAt:               Number page in which the term definition ends 
- volumeTitle:          Title of the Volume
- typeTerm:             Type of term [Topic| Articles]                                       
- year:                 Year of the edition
- volumeNum:            Volume number (e.g. 1)
- letters:              leters of the volume (A-B)
- part:                 Part of the volume (e.g 1)
- supplement:           Supplement's Title
- supplementsTo:        It suppelements to editions [1, 2, 3....]
- numberOfWords:        Number of words per term definition
- numberOfTerms:        Number of terms per page
- numberOfPages:        Number of pages per volume

In [170]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as pl
from rdflib import Graph, ConjunctiveGraph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery

In [171]:
import networkx as nx
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [172]:
text_embeddings = np.load('embeddings_1ed.npy')

In [173]:
import pickle
with open ('documents_1ed.txt', 'rb') as fp:
    documents = pickle.load(fp)
with open('terms_info_1ed.txt', 'rb') as fp2:
    terms_info = pickle.load(fp2)
with open('uris_1ed.txt', 'rb') as fp3:
    uris = pickle.load(fp3)

In [174]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [187]:
len(documents)

18117

In [175]:
uri="https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0"
term, definition=get_document(uri)

In [176]:
definition

'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.'

In [177]:
results_2=calculating_similarity_text(definition, text_embeddings)
results_2



{'https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0': ['1',
  '1773',
  '3',
  'PAISLEY',
  'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.',
  1.0000001],
 'https://w3id.org/eb/i/Article/992277653804341_144133903_PAISLEY_0': ['1',
  '1771',
  '3',
  'PAISLEY',
  'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.',
  1.0000001],
 'https://w3id.org/eb/i/Article/992277653804341_144133902_DUNS_0': ['1',
  '1771',
  '2',
  'DUNS',
  'a market-town of Scotland, twelve miles west of Berwick upon Tweed.',
  0.898082],
 'https://w3id.org/eb/i/Article/9929192893804340_144850367_DUNS_0': ['1',
  '1773',
  '2',
  'DUNS',
  'a market-town of Scotland, twelve miles west of Berwick upon Tweed.',
  0.898082],
 'https://w3id.org/eb/i/Article/9929192893804340_144850367_CULROSS_0': ['1',
  '1773',
  '2',
  'CULROSS',
  "a parliament town of Scotland, situated on the river Forth, about twenty-three miles north-west of Edinburgh : W. long. 

In [99]:
terms_info[index_df["id_2"].loc[0]]

['https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0',
 'PAISLEY',
 '1',
 '1773',
 '',
 '3',
 'M-Z']

In [152]:
documents[index_df["id_2"].loc[0]]

'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.'

In [153]:
similarities = cosine_similarity(text_embeddings)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[1.         0.19092174 0.23506945 ... 0.18361539 0.23539671 0.37386656]
 [0.19092174 1.         0.48545992 ... 0.53202856 0.6442368  0.4849234 ]
 [0.23506945 0.48545992 1.         ... 0.32033676 0.3483685  0.3656146 ]
 ...
 [0.18361539 0.53202856 0.32033676 ... 0.9999999  0.5364398  0.4388551 ]
 [0.23539671 0.6442368  0.3483685  ... 0.5364398  1.         0.63650733]
 [0.37386656 0.4849234  0.3656146  ... 0.4388551  0.63650733 0.9999999 ]]



In [154]:
similarities_sorted = similarities.argsort()
similarities_sorted

array([[10826,  7186,  5441, ..., 14941,  7527,     0],
       [ 7582,  8559, 16146, ..., 17178, 12601,     1],
       [12124, 10147,  2429, ...,  9304, 13278,     2],
       ...,
       [10527, 13686, 17853, ...,  4073,  8133, 18114],
       [13737,  1081,  7582, ..., 17574,  9456, 18115],
       [17160, 15748, 16644, ...,  8142, 18047, 18116]])

In [155]:
with open ('/Users/rosafilgueira/HW-Work/NLS-Fellowship/work/frances/web-app/models/similarities_1ed.txt', 'wb') as fp3:
     pickle.dump(similarities, fp3)
with open('/Users/rosafilgueira/HW-Work/NLS-Fellowship/work/frances/web-app/models/similarities_sorted_1ed.txt', 'wb') as fp4:
    pickle.dump(similarities_sorted, fp4)

In [156]:
with open ('/Users/rosafilgueira/HW-Work/NLS-Fellowship/work/frances/web-app/models/similarities_1ed.txt', 'rb') as fp:
    similarities = pickle.load(fp)
with open('/Users/rosafilgueira/HW-Work/NLS-Fellowship/work/frances/web-app/models/similarities_sorted_1ed.txt', 'rb') as fp2:
    similarities_sorted = pickle.load(fp2)

In [157]:
id_1 = []
id_2 = []
score = []
for index,array in enumerate(similarities_sorted):
    id_1.append(index)
    id_2.append(array[-2])
    score.append(similarities[index][array[-2]])
index_df = pd.DataFrame({'id_1' : id_1,
                          'id_2' : id_2,
                          'score' : score})

In [13]:
index_df

,id_1,id_2,score
0,0,7527,1.000000
1,1,12601,0.839797
2,2,13278,0.811463
3,3,4239,0.759265
4,4,55,0.981746
...,...,...,...
18112,18112,14396,0.812215
18113,18113,18068,0.993453
18114,18114,8133,0.826090
18115,18115,9456,0.845296


In [21]:
documents[7527]

'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.'

In [15]:
documents[7527]

'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.'

In [16]:
terms_info[7527]

['https://w3id.org/eb/i/Article/992277653804341_144133903_PAISLEY_0',
 'PAISLEY',
 '1',
 '1771',
 '',
 '3',
 'M-Z']

In [124]:
index_uri=uris.index("https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0")

In [125]:
index_uri

0

In [129]:
results={}
for i in range(-2, -12, -1):
    similar_index=similarities_sorted[index_uri][i]
    rank=similarities[index_uri][similar_index]
    results[uris[similar_index]]=[terms_info[similar_index][1],terms_info[similar_index][2], terms_info[similar_index][4], terms_info[similar_index][0], documents[similar_index], rank]
    
    
    
#terms_info.append([r["term"]["value"], r["enum"]["value"], r["year"]["value"], "" , r["vnum"]["value"], r["letters"]["value"]])
#[r_enum, r_year, r_vnum, r_term, r_definition, rank]

In [130]:
results

{'https://w3id.org/eb/i/Article/992277653804341_144133903_PAISLEY_0': ['1',
  '1771',
  '3',
  'PAISLEY',
  'a town of Scotland, in the county of Renfrew, six miles west of Glasgow.',
  1.0],
 'https://w3id.org/eb/i/Article/992277653804341_144133902_DUNS_0': ['1',
  '1771',
  '2',
  'DUNS',
  'a market-town of Scotland, twelve miles west of Berwick upon Tweed.',
  0.8980824],
 'https://w3id.org/eb/i/Article/9929192893804340_144850367_DUNS_0': ['1',
  '1773',
  '2',
  'DUNS',
  'a market-town of Scotland, twelve miles west of Berwick upon Tweed.',
  0.8980824],
 'https://w3id.org/eb/i/Article/9929192893804340_144850367_CULROSS_0': ['1',
  '1773',
  '2',
  'CULROSS',
  "a parliament town of Scotland, situated on the river Forth, about twenty-three miles north-west of Edinburgh : W. long. 3 0 34', and N. lat. 56°",
  0.8631866],
 'https://w3id.org/eb/i/Article/992277653804341_144133902_CULROSS_0': ['1',
  '1771',
  '2',
  'CULROSS',
  "a parliament town of Scotland, situated on the river 